# Seq2Seq Training

In [1]:
import os, datetime
import numpy as np
import joblib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence

## Collect HTML files

Load the HTML files

In [2]:
files = []

for f in os.listdir(os.path.join(os.getcwd(),'html')):
    fd = open(os.path.join(os.getcwd(),'html',f),'r')
    content = fd.read()
    files.append(content)
    fd.close()

Add a mock file as a concatentation of a previous one

In [3]:
files.append(files[0]*10)

## Preprocess the files

Define the padding and trimming parameters

In [4]:
padding='post'

maxlen=50
truncating='post'

Load the tokenizer

In [5]:
SQLi_tokenizer = joblib.load('ignore_tokenizer_20210318105336141483')

Convert the pages into token sequences:

In [6]:
tokens_list = SQLi_tokenizer.texts_to_sequences(files)

Pad and trim the sequences to fit them into an array:

In [7]:
tokens_array = keras.preprocessing.sequence.pad_sequences(tokens_list, padding=padding, maxlen=maxlen, truncating=truncating)

## Generating a target vector for teacher forcing

We set up a shifted (by -1) version of the input to allow the model to learn a step ahead.

In [8]:
target_array = np.zeros_like(tokens_array)
target_array[:,1:] = tokens_array[:,0:-1]

## Converting into one-hot encoding

Translating integer encoding into one-hot.

In [9]:
tokens_1hot_array = keras.utils.to_categorical(tokens_array)
target_1hot_array = keras.utils.to_categorical(target_array)

In [10]:
tokens_1hot_array.shape

(5, 50, 55)

## Defining Seq2Seq

Instantiating a VAE module built following [https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html).

In [11]:
latent_dim = 3
num_tokens = tokens_1hot_array.shape[2]

encoder_inputs = Input(shape=(None, num_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outpus, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,num_tokens))
decoder = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_,_ = decoder(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.summary()    

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 55)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 55)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 3), (None, 3 708         input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 3), (N 708         input_2[0][0]                    
                                                                 lstm[0][1]            

## Training the Seq2Seq

Instantiating and training the model

In [12]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([tokens_1hot_array,tokens_1hot_array],target_1hot_array,batch_size=5,epochs=1000)

Epoch 1/1000
1/1 [==============================] - 0s 2ms/step - loss: 4.0064 - accuracy: 0.0080
Epoch 2/1000
1/1 [==============================] - 0s 1ms/step - loss: 4.0040 - accuracy: 0.0320
Epoch 3/1000
1/1 [==============================] - 0s 2ms/step - loss: 4.0021 - accuracy: 0.0320
Epoch 4/1000
1/1 [==============================] - 0s 2ms/step - loss: 4.0006 - accuracy: 0.0320
Epoch 5/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.9993 - accuracy: 0.0320
Epoch 6/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.9980 - accuracy: 0.0440
Epoch 7/1000
1/1 [==============================] - 0s 973us/step - loss: 3.9968 - accuracy: 0.0440
Epoch 8/1000
1/1 [==============================] - 0s 5ms/step - loss: 3.9956 - accuracy: 0.0360
Epoch 9/1000
1/1 [==============================] - 0s 3ms/step - loss: 3.9945 - accuracy: 0.0360
Epoch 10/1000
1/1 [==============================] - 0s 5ms/step - loss: 3.9935 - accuracy: 0.0360
Epoch 11/1000
1/1

1/1 [==============================] - 0s 3ms/step - loss: 3.8953 - accuracy: 0.0920
Epoch 84/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.8934 - accuracy: 0.0880
Epoch 85/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.8916 - accuracy: 0.0880
Epoch 86/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.8897 - accuracy: 0.0880
Epoch 87/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.8878 - accuracy: 0.0880
Epoch 88/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.8859 - accuracy: 0.0880
Epoch 89/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.8840 - accuracy: 0.0880
Epoch 90/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.8821 - accuracy: 0.0880
Epoch 91/1000
1/1 [==============================] - 0s 3ms/step - loss: 3.8802 - accuracy: 0.0880
Epoch 92/1000
1/1 [==============================] - 0s 3ms/step - loss: 3.8783 - accuracy: 0.0880
Epoch 93/1000
1/1 [=====

1/1 [==============================] - 0s 2ms/step - loss: 3.7436 - accuracy: 0.1080
Epoch 165/1000
1/1 [==============================] - 0s 3ms/step - loss: 3.7418 - accuracy: 0.1080
Epoch 166/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.7401 - accuracy: 0.1080
Epoch 167/1000
1/1 [==============================] - 0s 7ms/step - loss: 3.7383 - accuracy: 0.1080
Epoch 168/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.7365 - accuracy: 0.1080
Epoch 169/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.7348 - accuracy: 0.1080
Epoch 170/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.7330 - accuracy: 0.1080
Epoch 171/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.7312 - accuracy: 0.1080
Epoch 172/1000
1/1 [==============================] - 0s 5ms/step - loss: 3.7294 - accuracy: 0.1080
Epoch 173/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.7276 - accuracy: 0.1080
Epoch 174/1000


1/1 [==============================] - 0s 2ms/step - loss: 3.5916 - accuracy: 0.1200
Epoch 247/1000
1/1 [==============================] - 0s 3ms/step - loss: 3.5897 - accuracy: 0.1200
Epoch 248/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.5878 - accuracy: 0.1200
Epoch 249/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.5860 - accuracy: 0.1200
Epoch 250/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.5841 - accuracy: 0.1200
Epoch 251/1000
1/1 [==============================] - 0s 5ms/step - loss: 3.5822 - accuracy: 0.1200
Epoch 252/1000
1/1 [==============================] - 0s 5ms/step - loss: 3.5803 - accuracy: 0.1200
Epoch 253/1000
1/1 [==============================] - 0s 4ms/step - loss: 3.5784 - accuracy: 0.1200
Epoch 254/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.5766 - accuracy: 0.1200
Epoch 255/1000
1/1 [==============================] - 0s 4ms/step - loss: 3.5747 - accuracy: 0.1200
Epoch 256/1000


1/1 [==============================] - 0s 2ms/step - loss: 3.4398 - accuracy: 0.1200
Epoch 329/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.4380 - accuracy: 0.1200
Epoch 330/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.4362 - accuracy: 0.1200
Epoch 331/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.4343 - accuracy: 0.1200
Epoch 332/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.4325 - accuracy: 0.1200
Epoch 333/1000
1/1 [==============================] - 0s 6ms/step - loss: 3.4307 - accuracy: 0.1200
Epoch 334/1000
1/1 [==============================] - 0s 6ms/step - loss: 3.4289 - accuracy: 0.1200
Epoch 335/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.4271 - accuracy: 0.1200
Epoch 336/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.4253 - accuracy: 0.1200
Epoch 337/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.4235 - accuracy: 0.1200
Epoch 338/1000


...

1/1 [==============================] - 0s 4ms/step - loss: 2.9131 - accuracy: 0.1960
Epoch 655/1000
1/1 [==============================] - 0s 914us/step - loss: 2.9117 - accuracy: 0.1960
Epoch 656/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.9102 - accuracy: 0.1960
Epoch 657/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.9088 - accuracy: 0.1960
Epoch 658/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.9074 - accuracy: 0.1960
Epoch 659/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.9059 - accuracy: 0.1960
Epoch 660/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.9045 - accuracy: 0.1960
Epoch 661/1000
1/1 [==============================] - 0s 906us/step - loss: 2.9031 - accuracy: 0.1960
Epoch 662/1000
1/1 [==============================] - 0s 919us/step - loss: 2.9016 - accuracy: 0.1960
Epoch 663/1000
1/1 [==============================] - 0s 871us/step - loss: 2.9002 - accuracy: 0.1960
Epoch 6

1/1 [==============================] - 0s 4ms/step - loss: 2.7997 - accuracy: 0.2120
Epoch 736/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.7984 - accuracy: 0.2120
Epoch 737/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.7970 - accuracy: 0.2120
Epoch 738/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.7957 - accuracy: 0.2120
Epoch 739/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.7943 - accuracy: 0.2120
Epoch 740/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.7930 - accuracy: 0.2120
Epoch 741/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.7916 - accuracy: 0.2120
Epoch 742/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.7903 - accuracy: 0.2120
Epoch 743/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.7890 - accuracy: 0.2120
Epoch 744/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.7876 - accuracy: 0.2120
Epoch 745/1000


1/1 [==============================] - 0s 6ms/step - loss: 2.6927 - accuracy: 0.2560
Epoch 818/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.6914 - accuracy: 0.2560
Epoch 819/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.6901 - accuracy: 0.2560
Epoch 820/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.6889 - accuracy: 0.2560
Epoch 821/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.6876 - accuracy: 0.2600
Epoch 822/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.6864 - accuracy: 0.2560
Epoch 823/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.6851 - accuracy: 0.2600
Epoch 824/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.6838 - accuracy: 0.2600
Epoch 825/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.6826 - accuracy: 0.2600
Epoch 826/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.6813 - accuracy: 0.2600
Epoch 827/1000


1/1 [==============================] - 0s 4ms/step - loss: 2.5910 - accuracy: 0.2720
Epoch 900/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.5898 - accuracy: 0.2720
Epoch 901/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.5886 - accuracy: 0.2720
Epoch 902/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.5874 - accuracy: 0.2720
Epoch 903/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.5861 - accuracy: 0.2720
Epoch 904/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.5849 - accuracy: 0.2720
Epoch 905/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.5837 - accuracy: 0.2720
Epoch 906/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.5825 - accuracy: 0.2720
Epoch 907/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.5813 - accuracy: 0.2720
Epoch 908/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.5801 - accuracy: 0.2720
Epoch 909/1000


Epoch 981/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.4942 - accuracy: 0.3160
Epoch 982/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.4930 - accuracy: 0.3160
Epoch 983/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.4919 - accuracy: 0.3160
Epoch 984/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.4908 - accuracy: 0.3160
Epoch 985/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.4896 - accuracy: 0.3160
Epoch 986/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.4885 - accuracy: 0.3160
Epoch 987/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.4873 - accuracy: 0.3160
Epoch 988/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.4862 - accuracy: 0.3160
Epoch 989/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.4851 - accuracy: 0.3160
Epoch 990/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.4839 - accuracy: 0.3160


In [13]:
encoder_model = keras.Model(encoder_inputs,encoder_states)
encoder_model.predict(tokens_1hot_array[0:4,:,:])

[array([[-0.36717224, -0.97918844, -0.38257056],
        [-0.36015296, -0.9704888 , -0.32849747],
        [-0.41673118, -0.9841541 , -0.3286019 ],
        [-0.44060656, -0.9802002 , -0.3315617 ]], dtype=float32),
 array([[-32.142063, -43.924225, -33.829628],
        [-31.659052, -43.622475, -33.54474 ],
        [-30.938274, -43.512222, -33.616623],
        [-32.059254, -43.75573 , -33.6907  ]], dtype=float32)]

**NOTE**: interestingly, with length $200$ and training using *optimizer='rmsprop'* or *optimizer='adam'* and *loss='categorical_crossentropy'*, the model seems often to converge to a solution where every input is mapped to a vector of 0s. This is probably due to the fact that most of the inputs are padded with zeros, so outputting all zeros achieves a reasonably low loss. This issue may be worked around reducing the padding (e.g.: $50$)

## Checking the Seq2Seq

In [14]:
encoder_model = keras.Model(encoder_inputs,encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_input = [decoder_state_input_h,decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(decoder_inputs, initial_state=decoder_states_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs]+decoder_states_input,
                           [decoder_outputs]+decoder_states)

In [15]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    print(states_value)

    target_seq = np.zeros((1, 1, num_tokens))
    target_seq[0, 0, 0] = 1.

    decoded_sentence = []
    for _ in range(maxlen):
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_sentence.append(sampled_token_index)

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [16]:
decode_sequence(tokens_1hot_array[0:1,:,:])

[array([[-0.36717224, -0.97918844, -0.38257056]], dtype=float32), array([[-32.142063, -43.924225, -33.829628]], dtype=float32)]


[13,
 14,
 13,
 14,
 13,
 14,
 13,
 14,
 13,
 14,
 13,
 14,
 16,
 16,
 16,
 17,
 18,
 18,
 18,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6]

In [17]:
decode_sequence(tokens_1hot_array[3:4,:,:])

[array([[-0.44060656, -0.9802002 , -0.3315617 ]], dtype=float32), array([[-32.059254, -43.75573 , -33.6907  ]], dtype=float32)]


[13,
 14,
 13,
 14,
 13,
 14,
 13,
 14,
 13,
 14,
 13,
 14,
 16,
 16,
 16,
 17,
 18,
 18,
 18,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6]

In [18]:
decode_sequence(tokens_1hot_array[3:4,:,:]) == decode_sequence(tokens_1hot_array[1:2,:,:])

[array([[-0.44060656, -0.9802002 , -0.3315617 ]], dtype=float32), array([[-32.059254, -43.75573 , -33.6907  ]], dtype=float32)]
[array([[-0.36015296, -0.9704888 , -0.32849747]], dtype=float32), array([[-31.659052, -43.622475, -33.54474 ]], dtype=float32)]


True

## Saving the Seq2Seq

Saving the model

In [19]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
model.save('ignore_seq2seq_'+timestamp)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ignore_seq2seq_20210321114421788288/assets
